In [ ]:
import re
import nltk
from  nltk.stem import PorterStemmer
from nltk.corpus import twitter_samples,stopwords
from string import punctuation
from nltk.tokenize import TweetTokenizer
nltk.download('twitter_samples')
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
def preprocess_tweete(corbus,tokenizer,steeming,stop_words):
    clean_corbus=[]
    for index,tweet in enumerate (corbus):
        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '',tweet )

        # remove hyperlinks
        tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)

        # remove hashtags
        # only removing the hash # sign from the word
        tweet = re.sub(r'#', '', tweet)
        
        tokenzie_tweet=tokenizer(tweet)
        
        clean_tweet=[]
        for token in tokenzie_tweet:
            if token not in stop_words and token not in punctuation:
                clean_tweet.append(steeming(token))

        
        
        clean_corbus.append(clean_tweet)
    return clean_corbus

        

In [ ]:
def freq_dict(clean_tweet,labels):
    freq_dict={}
    for label,tweet in zip(labels,clean_tweet):
        for word in tweet:
            freq_dict[(word,label)]=freq_dict.get((word,label),0)+1
    return freq_dict

In [ ]:
def create_training_matrix(tweets,frequence_dict):
    matrix=np.zeros([len(tweets),3])
    matrix[:,0]=1

    for index ,tweet in enumerate(tweets):
        positive=0
        negative=0
        for word in tweet:
            positive+=frequence_dict.get((word,1),0)
            negative+=frequence_dict.get((word,0),0)
            
        matrix[index,1]=positive
        matrix[index,2]=negative
        
    
    return matrix

In [ ]:
def sigmoid(X,theta):
    z=np.matmul(X,theta)
    return 1/(1+(np.exp(-z)))

In [ ]:
def training(X,Y,theta,alpha,iteration):
    m=len(Y)
    cost=[]
    for  i in range(iteration):
        
        h=sigmoid(X,theta)
        

        loss=-1/m * (np.squeeze(np.matmul(Y.T , np.log(h))) + np.squeeze(np.matmul((1 - Y).T , np.log(1-h))))
        theta=theta - alpha / m * np.matmul(X.T,(h-Y[:,np.newaxis]))
        cost.append(loss)
        
        
    return theta,cost

In [ ]:
def compute_accuracy(X,Y,theta):
    z=sigmoid(X,theta)
    preidect=z>.5
    accuracy=sum(preidect==Y[:,np.newaxis])/len(Y)
    return accuracy
    

In [ ]:
def pridect(tweet,theta,freq):
    tweet=np.array([tweet])
    tweet=preprocess_tweete(tweet,tokenizer.tokenize,stemming.stem,stop_words)
    

    tweet=create_training_matrix(tweet,freq)
    
    predict=sigmoid(tweet,theta)
    if predict >0.5:
        analysis='Positive'
    else:
        analysis='Negative'
        
    return analysis
    
    

In [ ]:
positive_tweets=twitter_samples.strings('positive_tweets.json')
negative_tweets=twitter_samples.strings('negative_tweets.json')

In [ ]:
print('example on positive tweet :-',positive_tweets[0],'\n')
print('example on positive tweet:-',negative_tweets[0])

In [ ]:
x=[len(positive_tweets),len(negative_tweets)]
y=['positive_tweet','negative_tweet']
data_shape=plt.pie(x,labels=y,startangle=90,shadow=True,colors=['c','r'])

In [ ]:
label=np.append(np.ones(len(positive_tweets)),np.zeros(len(negative_tweets)))
all_tweets=np.append(positive_tweets,negative_tweets)

In [ ]:
stemming=PorterStemmer()
tokenizer=TweetTokenizer(   preserve_case=False,
    reduce_len=True,
    strip_handles=True)
stop_words=stopwords.words('english')

In [ ]:
clean_tweete=preprocess_tweete(all_tweets,tokenizer.tokenize,stemming.stem,stop_words)

In [ ]:
freq=freq_dict(clean_tweete,label)

In [ ]:
matrix=create_training_matrix(clean_tweete,freq)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(matrix,label,test_size=.2)

In [ ]:
theta,cost=training(X_train,y_train,np.zeros((3,1)),1e-8,700)
theta

In [ ]:
plt.scatter(cost[0:700],[i for i in range(700)],)
plt.xlabel('Loss')
plt.ylabel('Nuber of iteration')

In [ ]:
accuracy=compute_accuracy(X_test,y_test,theta)

In [ ]:
accuracy

In [ ]:
print(pridect('i am sad'))
print(pridect('nice to meet you'))